In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

In [8]:
#Reading input images and putting into numpy array
data = []
labels = []

height = 30
width = 30
channels = 3
classes = 43
n_inputs = height * width * channels

for i in range(classes):
    path='./data/GTSRB/train/{0}/'.format(i)
    #print(path)
    Class = os.listdir(path)
    for img in Class:
        try:
            image = cv2.imread(path+img)
            image_from_array = Image.fromarray(image, 'RGB')
            resized_image = image_from_array.resize((height, width))
            data.append(np.array(resized_image))
            labels.append(i)
        except AttributeError:
            pass

Cells = np.array(data)
labels = np.array(labels)

In [9]:
# Randomize the order of the input images
s = np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells = Cells[s]
labels = labels[s]

In [11]:
# Spliting the images into train and validation sets
(X_train, X_val) = Cells[(int)(0.2*len(labels)):], Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
(y_train, y_val) = labels[(int)(0.2*len(labels)):], labels[:(int)(0.2*len(labels))]

#Using one hote encoding for the train and validation labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

In [19]:
# Define DNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [20]:
# Compile the model
model.compile(
    loss = 'categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [21]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        2432      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 5, 64)         

In [26]:
# Train the model
epochs = 30
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs)
validation_data=(X_val, y_val)

Epoch 1/30
981/981 [==============================] - 34s 35ms/step - loss: 0.1188 - accuracy: 0.9636
Epoch 2/30
981/981 [==============================] - 38s 39ms/step - loss: 0.0976 - accuracy: 0.9701
Epoch 3/30
981/981 [==============================] - 39s 40ms/step - loss: 0.0782 - accuracy: 0.9747
Epoch 4/30
981/981 [==============================] - 45s 46ms/step - loss: 0.0670 - accuracy: 0.9793
Epoch 5/30
981/981 [==============================] - 43s 44ms/step - loss: 0.0698 - accuracy: 0.9794
Epoch 6/30
981/981 [==============================] - 43s 44ms/step - loss: 0.0545 - accuracy: 0.9826
Epoch 7/30
981/981 [==============================] - 45s 46ms/step - loss: 0.0576 - accuracy: 0.9830
Epoch 8/30
981/981 [==============================] - 41s 42ms/step - loss: 0.0495 - accuracy: 0.9854
Epoch 9/30
981/981 [==============================] - 38s 39ms/step - loss: 0.0474 - accuracy: 0.9854
Epoch 10/30
981/981 [==============================] - 39s 40ms/step - loss: 0.045

In [43]:
model.save('./model/GTSRB-1-30epochs.h5')

Dataset: https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign